# Phase B: Model Diagnostics & Advanced Feature Engineering
**Project:** Real Estate Price Prediction (Queretaro, Mexico)
**Objective:** Diagnose the econometric anomalies found in the Baseline BigQuery models and implement feature engineering to resolve multicollinearity and endogeneity issues.

## 1. Baseline Model "Post-Mortem" Analysis
In Phase A, we trained two baseline models using BigQuery ML:
1.  **Model A (Linear):** Target = `price` ($R^2 \approx 0.645$)
2.  **Model B (Log-Linear):** Target = `ln(price)` ($R^2 \approx 0.623$)

While the $R^2$ results successfully replicated the explanatory power of the reference paper (Guanajuato study), the coefficient analysis revealed three **econometric paradoxes** that must be addressed before model tuning:

| model\_name | feature\_name | coefficient | standard\_error | p\_value | significance\_level |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Model A \(Linear\) | \_\_INTERCEPT\_\_ | -3569918.99 | 368817.35 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_bathrooms | 688079.31 | 50238.23 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_bedrooms | -492347.23 | 65832.46 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_crime\_consolidated | 5756.01 | 399.49 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_crime\_homicide | 60931.28 | 33203.45 | 0.066 | \* Marginally Significant |
| Model A \(Linear\) | feat\_dist\_center | -122.93 | 17.66 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_dist\_park | -244.36 | 44.65 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_dist\_supermarket | 238.07 | 29.54 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_has\_garden | 354276.87 | 101621.16 | 0.0008 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_m2\_constructed | 22112.04 | 282.79 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_parking\_spots | 486444.87 | 28637.46 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | \_\_INTERCEPT\_\_ | 14.0734 | 0.0365 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_bathrooms | 0.1811 | 0.0051 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_bedrooms | -0.0272 | 0.0066 | 0.0001 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_crime\_consolidated | 0.0005 | 0 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_crime\_homicide | -0.0121 | 0.0033 | 0.0004 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_dist\_center | 0 | 0 | 0.7211 | Not Significant \(Likely Noise\) |
| Model B \(Log-Linear\) | feat\_dist\_park | -0 | 0 | 0.1956 | Not Significant \(Likely Noise\) |
| Model B \(Log-Linear\) | feat\_dist\_supermarket | 0 | 0 | 0.0874 | \* Marginally Significant |
| Model B \(Log-Linear\) | feat\_has\_garden | 0.1832 | 0.0102 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_m2\_constructed | 0.002 | 0 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_parking\_spots | 0.0393 | 0.0029 | 0 | \*\*\* Highly Significant |


### A. The "Negative Bedroom" Paradox
* **Observation:** The coefficient for `feat_bedrooms` was **negative** (-492,347 MXN in Linear model).
* **Interpretation:** The model suggests that adding a bedroom *reduces* the house value by half a million pesos, holding other factors constant.
* **Diagnosis:** High probability of **Multicollinearity**. Since `feat_m2_constructed` (size) is already in the model, `bedrooms` and `size` are fighting to explain the same variance. A small house with many rooms implies tiny, lower-value rooms.

### B. The "Crime Value" Paradox
* **Observation:** The coefficient for `feat_crime_consolidated` was **positive** (+5,756 MXN).
* **Interpretation:** The model implies that higher crime rates are associated with *higher* property values.
* **Diagnosis:** **Endogeneity / Omitted Variable Bias**. Crime is likely acting as a proxy for **economic activity**. Wealthy areas attract more opportunistic crime than undeveloped areas. The model is confusing "wealthy target" with "valuable location."

### C. The "Vanishing Distance" Effect
* **Observation:** In the Log-Linear Model (Model B), distance variables (Park, Center) became statistically insignificant ($p\text{-value} > 0.05$).
* **Diagnosis:** The logarithmic transformation of the price compressed the variance so much that the subtle linear effect of distance was lost. This suggests the relationship between distance and price might be non-linear or strictly local (e.g., only matters within 500m).

---
## 2. Hypothesis & Action Plan
To resolve these issues and improve $R^2$ > 0.70, we will test the following hypotheses:

1.  **H1 (Multicollinearity):** Removing `bedrooms` or creating an interaction term (e.g., `bathrooms_per_bedroom`) will stabilize the coefficients of physical characteristics.
2.  **H2 (Crime Unbundling):** Instead of a consolidated crime rate, we will separate **Violent Crime** (expected negative impact) from **Property Crime** (proxy for wealth) to isolate the true "fear factor."
3.  **H3 (Rich Features):** Incorporating specific amenities (Schools, Hospitals) will capture the variance currently attributed to "noise" in the Baseline.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Visualization Settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 8]
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Initialize BigQuery Client
client = bigquery.Client()

print("Libraries loaded. Ready for diagnostics.")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

## 3. Data Loading: Unleashing the Full OBT (One Big Table)

In Phase A (Baseline), we intentionally restricted our analysis to a limited subset of variables to strictly replicate the methodology of the reference paper (Guanajuato study).

**Transition to Phase B:**
Now that the baseline is established, we proceed to unlock the full potential of our **One Big Table (OBT)**. Unlike the original study, our dataset includes a richer set of granular features collected from multiple sources.

**Key Additions in this Phase:**
* **Crime Granularity:** Instead of a consolidated sum, we access specific crime types (Homicide, Burglary, Violence, Vehicle Theft) to test the "Unbundling" hypothesis.
* **Expanded Amenities:** We include previously unused proximity features (Schools, Hospitals, etc.) to capture value drivers that were treated as "noise" in the baseline.
* **Full Physical Specs:** Detailed breakdown of property characteristics (e.g., Half-bathrooms, Age of construction).

We will now query the full `view_training_data_cleaned` to perform the advanced diagnostic.

In [ ]:
# Query to fetch all potential features + target
query = """
SELECT
    -- Target
    target_price,

    -- Physical Features
    feat_bedrooms,
    feat_bathrooms,
    feat_half_bathrooms,
    feat_parking_spots,
    feat_m2_constructed,
    feat_m2_land,
    feat_year_built,

    -- Geographic Features (Distances)
    feat_dist_mall,
    feat_dist_park,
    feat_dist_industrial,
    feat_dist_center,

    -- Crime Features (Raw Counts)
    feat_crime_homicide,
    feat_crime_burglary,
    feat_crime_violence,
    feat_crime_vehicle

FROM `real-estate-qro.queretaro_data_marts.view_training_data_cleaned`
WHERE target_price IS NOT NULL
"""

# Load to DataFrame
df = client.query(query).to_dataframe()

# Quick sanity check
print(f"Dataset Shape: {df.shape}")
df.head()

## 4. Diagnosing Multicollinearity (The "Bedroom" Issue)

To confirm why the bedroom coefficient is negative, we need to check how strongly the physical variables relate to each other.

**Tools used:**
1.  **Correlation Matrix:** Visual inspection of relationships.
2.  **VIF (Variance Inflation Factor):** A statistical measure quantifying how much the variance of a coefficient is inflated due to multicollinearity.
    * *Rule of Thumb:* VIF > 5 indicates high multicollinearity. VIF > 10 requires immediate action (dropping variables).

In [ ]:
# --- A. Correlation Matrix Heatmap ---
# Calculate correlation matrix
corr_matrix = df[['target_price', 'feat_bedrooms', 'feat_bathrooms',
                  'feat_m2_constructed', 'feat_parking_spots']].corr()

# Plot
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # Hide upper triangle

sns.heatmap(corr_matrix,
            mask=mask,
            annot=True,
            fmt=".2f",
            cmap='RdBu_r',
            vmax=1,
            vmin=-1,
            linewidths=.5)

plt.title('Correlation: Physical Features vs Price', fontsize=14)
plt.show()

# --- B. Variance Inflation Factor (VIF) Calculation ---
# Select only numeric features for VIF check (excluding target)
features_for_vif = df[['feat_bedrooms', 'feat_bathrooms',
                       'feat_m2_constructed', 'feat_parking_spots']].dropna()

# Create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = features_for_vif.columns
vif_data["VIF"] = [variance_inflation_factor(features_for_vif.values, i)
                   for i in range(len(features_for_vif.columns))]

print("\n--- Variance Inflation Factor (VIF) Results ---")
print(vif_data.sort_values(by="VIF", ascending=False))